In [1]:
import pandas as pd

import clearml

In [3]:
task = clearml.Task.get_task("9b9a0a7b1d504caab444b37c648a4090")
artifact = task.artifacts["predications"].get()

In [6]:
df = pd.read_csv(artifact)
df.sample(n=5)

,filename,words,preds
2392,image_545952.jpg,4350,4350
8736,image_38991.jpg,69999,69999
3142,18999_image_91796.jpg,18999,18999
15447,399_image_96637.jpg,399,399
7495,image_665045.jpg,14499,14499


In [8]:
failed_photo = df[df["words"] != df["preds"]]
truth_photo = df[df["words"] == df["preds"]]

In [12]:
from pathlib import Path
import shutil
source = Path("../data/val/images")
target = Path("./results/test_cyrillic_g2_8000_Adadelta_with_GradScaler_failed_photo")
target.mkdir(exist_ok=True,parents=True)
for _,row in failed_photo.iterrows():
    shutil.copy(source/row["filename"],target/row["filename"])